In [ ]:
import matplotlib.font_manager as fm

!apt-get -qq -y install fonts-nanum > /dev/null
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild()

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False
	
# 한글 폰트 설정
path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
font_name = fm.FontProperties(fname=path, size=18).get_name()
plt.rc('font', family=font_name)
fm._rebuild()

In [ ]:
import os
import numpy as np
import pandas as pd
import sys

%matplotlib inline
import matplotlib.pylab as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (20, 10)

path = '/content/drive/MyDrive/data/LPOINT/'

In [ ]:
file_list = os.listdir(path)
file_list

In [ ]:
'''
DEMO - 고객 데모
PDDE - 유통사 상품 구매 내역
PD_CLAC - 상품 분류 정보
BR - 점포 정보
COP_U -  제휴사 이용 정보
LPAY - 엘페이 이용
'''

demo = pd.read_csv(path + file_list[0])
pdde = pd.read_csv(path + file_list[4])                                            
cop_u = pd.read_csv(path + file_list[2])
pd_clac = pd.read_csv(path + file_list[1])
br = pd.read_csv(path + file_list[3])
lpay = pd.read_csv(path + file_list[5])

In [ ]:
pd_ = pd.merge(pd_clac, pdde) 
pd_.cop_c = pd_.pd_nm
pd_ = pd_.drop(['br_c', 'pd_nm','clac_hlv_nm','clac_mcls_nm'], axis=1)
data1 = pd.merge(demo,pd.concat([cop_u, pd_]))
data1['buy_ct'] = data1['buy_ct'].fillna(1)
data1 = data1.drop(['vst_dt','pd_c'],axis= 1)
data1.columns = ['고객번호','성별','연령대','거주지대분류코드','영수증번호','상품코드','점포코드','채널구분','구매일자','구매시간','구매금액','구매수량']

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
data1['상품코드'].unique()

In [ ]:
# 추천에 무의미하다고 생각한 상품들은 제거하고 진행 

data = data1[(data1['상품코드'] != '봉투 보증금') & (data1['상품코드'] != '종량제봉투') & (data1['상품코드'] != '임대매출')]

In [ ]:
agg_data1 = data1.groupby(['구매시간', '연령대', '성별'], as_index=False)[['고객번호']].count().rename(columns={'고객번호' : '구매건수'})
male = agg_data1[agg_data1['성별']=='남성']
female = agg_data1[agg_data1['성별']=='여성']

# 남성
plt.subplot(2,1,1)
plt.title('시간대별 남성고객의 구매 분포', fontsize = 14)
sns.barplot(data=male, x='구매시간', y='구매건수', hue='연령대')
plt.xlabel('구매시간', fontsize = 12)

# 여성
plt.subplot(2,1,2)
plt.title('시간대별 여성고객의 구매 분포', fontsize = 14)
sns.barplot(data=female, x='구매시간', y='구매건수', hue='연령대')
plt.xlabel('구매시간', fontsize = 12)

plt.show()


# 40대가 가장 많이 이용했으며, 주 이용 시간은 15시 ~ 19시이다.
# 성비로는 여성 고객이 대략 2배 정도 더 많이 이용 하였다.

In [ ]:
def flat_columns(df):
    df.columns = ['_'.join(x) for x in df.columns.to_flat_index()]
    return df

agg_df = (flat_columns(data1.groupby(['상품코드'], as_index=False)[['구매수량', '구매금액']].agg({ '구매수량' : ['count', 'max', 'min'],
           '구매금액' : ['max', 'min', 'mean', 'std']})).fillna(0)
).rename(columns = {'상품코드_' : '상품코드', '구매수량_count' : '총판매건수',
                    '구매수량_max' : '최다동시판매수', '구매수량_min' : '최소동시판매수',
                    '구매금액_max' : '판매최고가', '구매금액_min' : '판매최저가', 
                    '구매금액_mean' : '판매평균가', '구매금액_std' : '판매표준편차'}).set_index('상품코드')

In [ ]:
agg_df.tail()

In [ ]:
# 상품의 퀄리티에 비해 가격이 싸다고 느낀 데이터들이 존재

agg_df[agg_df['판매최저가'] < 100].tail()

In [ ]:
# 금액이 1000 이하의 데이터들을 이상치로 여겨 제거

data = data1[data1['구매금액'] > 1000]

In [ ]:
data.to_csv("data.csv")

In [ ]:
# 특정 상품들이 채널별로 유통의 차이가 명확하게 식별되어
# 가장 명확한 상품들을 구분하여 유통채널별로 상위 10개의 상품을 조회

agg_data = pd.pivot_table(data, values='구매수량', index='상품코드', columns='채널구분', aggfunc=np.sum).fillna(0)
agg_data['유통차이'] = agg_data[1] - agg_data[2] # 양수이면 오프라인 판매량이 더 많은경우, 음수이면 온라인 판매량이 더 많은 경우에 해당

offline_data = agg_data[agg_data['유통차이'] > 0].abs().sort_values(by='유통차이', ascending=False)[:10]
online_data = agg_data[agg_data['유통차이'] < 0].abs().sort_values(by='유통차이', ascending=False)[:10]

# 온라인 top10
fig, ax = plt.subplots(figsize=(20, 10))
plt.subplot(2,1,1)
sns.barplot(data=online_data, x=online_data.index, y='유통차이')
plt.xlabel('온라인 상위 상품')
plt.ylabel('온라인 - 오프라인')

# 오프라인 top10
plt.subplot(2,1,2)
sns.barplot(data=offline_data, x=offline_data.index, y='유통차이')
plt.xlabel('오프라인 상위 상품')
plt.ylabel('오프라인 - 온라인')

# 온라인 상위 상품은 대부분 냉동, 냉장 식품 또는 온라인으로 구매가 용이한 상품이 존재한다.
# 오프라인의 경우 인근 마트에서 간단한게 살 수 있는 상품이 존재한다.

In [ ]:
# 구매일자를 datatime형으로 변환하기위해 '/'를 년, 월, 일 중간에 삽입해준다.

def to_date(dt):
    t_dt = dt[:4]+'/'+dt[4:6]+'/'+dt[6:]
    return t_dt

data['구매일자'] = data['구매일자'].astype('str').apply(to_date) 

# 대부분이 2021년도의 구매데이터인것으로 확인된다. 
# 고객별 구매한 날짜를 기준으로 편차를 확인하기 위해 2021/01/01을 1로 기준을 잡아서 2021/12/31이 365가 되게 구매 날짜 컬럼을 만든다.

data = data.assign(구매날짜 = (pd.to_datetime(data['구매일자'])-pd.to_datetime('2021/01/01')).astype(str))
def to_num(dt):
    t_num = dt[:-5]
    return t_num
data['구매날짜'] = data['구매날짜'].apply(to_num).astype('int')
data['구매날짜'] = data['구매날짜'] + 1 # 0부터 시작하기에 1씩 더한다.

In [ ]:
# 구매일자를 datatime형태로 변환한 뒤, weekday()함수를 활용하여 요일을 컬럼을 생성
# 0: 월요일 ~ 6: 일요일

data['구매일자'] = pd.to_datetime(data['구매일자'])

def t_week(dt):
    tw = dt.weekday()
    return tw

data = data.assign(구매요일 = data['구매일자'].apply(t_week))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 성별, 연령대 컬럼 Label Encoding 진행
# 성별: 0(남성), 1(여성)
# 연령대: 0(20대) ~ 5(70대)

data.iloc[:, 1:3] = \
data.iloc[:, 1:3].apply(LabelEncoder().fit_transform)

In [ ]:
# 각 요일별 구매 수량을 확인

data_bydate = data.groupby(['구매요일'])['구매수량'].agg('count')
data_bydate.plot(kind = 'bar')
plt.xlabel('구매요일', fontsize=15)
plt.xticks(fontsize=14, rotation = 0)

# 금요일에 조금 올라가면서, 토요일에 급격하게 늘어나는 것을 확인할 수 있다.

In [ ]:
# 꾸준히 이용한 고객을 구매일자의 표준편차가 큰 고객으로 가정하여 고객별 충성도 계산
# 그리고 표준편차에 1년에 구매 횟수의 비율을 곱하여 충성도를 계산하였다.

loyal = pd.DataFrame((data.groupby('고객번호').구매날짜.agg(np.std).fillna(0) * (data.groupby('고객번호').고객번호.agg('count')/365))).reset_index()
loyal.columns = ['고객번호','충성도']
data = pd.merge(data,loyal)

In [ ]:
# 충성도에 대한 상관관계를 보면, 컬럼과의 연관성이 없다는 것을 볼 수 있다.

data.corr().충성도

In [ ]:
# 성별과 연령대를 기준으로 t-sne 시각화 2개 진행
# 데이터 양이 크기에 약 23000개로 샘플링
# stratify는 층화 추출 의미

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

# 시각화에 영향을 미칠만한 컬럼 선정 
data_tsne = data[['성별', '연령대', '거주지대분류코드', '상품코드', '채널구분', '구매시간', '구매금액']]

tsne_sample_sex, test = train_test_split(data_tsne, test_size = 0.995, random_state = 42, stratify = data_tsne['성별'])
tsne_sample_age, test = train_test_split(data_tsne, test_size = 0.995, random_state = 42, stratify = data_tsne['연령대'])
print(tsne_sample_sex.shape, tsne_sample_age.shape)

In [ ]:
# 거주지, 상품 - one hot encoding

tsne_sample_sex = pd.get_dummies(tsne_sample_sex, columns = ['거주지대분류코드', '상품코드'])
tsne_sample_age = pd.get_dummies(tsne_sample_age, columns = ['거주지대분류코드', '상품코드'])

In [ ]:
from sklearn.manifold import TSNE
plt.rc('axes', unicode_minus=False)

tsne = TSNE(random_state = 42)
tsne_sex = tsne.fit_transform(tsne_sample_sex)

tsne_df = pd.DataFrame(tsne_sex, columns = ['component 0', 'component 1'])
tsne_df['sex'] = tsne_sample_sex['성별'].values
tsne_df_0 = tsne_df[tsne_df['sex'] == 0]
tsne_df_1 = tsne_df[tsne_df['sex'] == 1]

plt.figure(figsize=(10,10))
plt.scatter(tsne_df_0['component 0'], tsne_df_0['component 1'], color = 'red', label = 'male')
plt.scatter(tsne_df_1['component 0'], tsne_df_1['component 1'], color = 'blue', label = 'female')
 
plt.xlabel("t-SNE 특성 0")
plt.ylabel("t-SNE 특성 1")

In [ ]:
plt.rc('axes', unicode_minus=False)
tsne_age = tsne.fit_transform(tsne_sample_age)

tsne_df = pd.DataFrame(tsne_age, columns = ['component 0', 'component 1'])
tsne_df['age'] = tsne_sample_sex['연령대'].values
tsne_df_0 = tsne_df[tsne_df['age'] == 0]
tsne_df_1 = tsne_df[tsne_df['age'] == 1]
tsne_df_2 = tsne_df[tsne_df['age'] == 2]
tsne_df_3 = tsne_df[tsne_df['age'] == 3]
tsne_df_4 = tsne_df[tsne_df['age'] == 4]
tsne_df_5 = tsne_df[tsne_df['age'] == 5]

plt.figure(figsize=(10,10))

plt.scatter(tsne_df_0['component 0'], tsne_df_0['component 1'],  label = '20대')
plt.scatter(tsne_df_1['component 0'], tsne_df_1['component 1'],  label = '30대')
plt.scatter(tsne_df_2['component 0'], tsne_df_2['component 1'],  label = '40대')
plt.scatter(tsne_df_3['component 0'], tsne_df_3['component 1'],  label = '50대')
plt.scatter(tsne_df_4['component 0'], tsne_df_4['component 1'],  label = '60대')
plt.scatter(tsne_df_5['component 0'], tsne_df_5['component 1'],  label = '70대')
 
plt.xlabel("t-SNE 특성 0")
plt.ylabel("t-SNE 특성 1")